<a href="https://colab.research.google.com/github/voodoohop/colabasaservice/blob/master/colabs/deep-daze-.iframe.experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#@title
from IPython.display import IFrame

IFrame(src='https://voodoohop.github.io/colabasaservice/', width=700, height=600)

In [4]:
!pip install deep-daze stomp.py imageio-ffmpeg --upgrade

Requirement already up-to-date: deep-daze in /usr/local/lib/python3.7/dist-packages (0.10.2)
Requirement already up-to-date: stomp.py in /usr/local/lib/python3.7/dist-packages (7.0.0)
     |████████████████████████████████| 26.9MB 1.3MB/s 


In [6]:
from subprocess import Popen, PIPE, STDOUT
from sys import stdout
from glob import glob
import os
import stomp
import json
import itertools
from base64 import b64encode
from time import sleep
import requests
import os
from mimetypes import guess_type 

flatten = itertools.chain.from_iterable

TEXT="Petite sexy and horny photorealistic"

DEFAULT_PARAMS='--lower_bound_cutout 0.4 --image-width=256 --iterations=500 --epochs=2 --save-every=2 --save-progress=True --open_folder=False --overwrite=True --save_video=True'

conn = stomp.Connection([('b-4e89df1f-8226-4cc4-a518-4e1ac0023c97-1.mq.eu-central-1.amazonaws.com', 61614)], use_ssl=True)
conn.connect('guest', 'iamcolabguest', wait=True)



def run(text, params=DEFAULT_PARAMS, watch_media_paths=["./*.jpg","./*.mp4"]):
  !rm *.jpg
  !rm *.png

  cmd = f'imagine {params}'
  
  p = Popen(cmd, stdout = PIPE, 
          stderr = STDOUT, shell = True)

  found_media_files=[]
  while True:
    line = p.stdout.readline()#
    if not line: break

    stdout.write(line)
    conn.send(body=line.decode("utf-8"), destination='/topic/colabOut')
    new_found_media_files = get_matching_files(watch_media_paths)
    new_found_media_files.sort(key=os.path.getmtime)
    added_media_files = list(set(new_found_media_files)-set(found_media_files))
    found_media_files = new_found_media_files

    if len(added_media_files) > 0:
      for added in added_media_files:
        sleep(0.1)
        if len(added) > 0:
          filename = os.path.basename(added)
          mimetype, _ = guess_type(filename)
          print("Adding:", added,"type",mimetype)
          
          with open(added,"rb") as media_file:
            added_media = media_file.read()

            base64_encoded_media = f"data:{mimetype};base64,{b64encode(added_media).decode('ascii')}"
            headers = { 
                "type": mimetype,
                "filename": filename,
                "params": params

            }
          conn.send(body=base64_encoded_media,headers=headers,destination='/topic/colabMediaOut')

def get_matching_files(search_paths):
  return list(flatten([glob(search_path) for search_path in search_paths]))
run()
conn.disconnect()



rm: cannot remove '*.png': No such file or directory
epochs:   0%|          | 0/2 [00:00<?, ?it/s]
Adding: ./Petite_sexy_and_horny_photorealistic.mp4 type video/mp4
                                             


KeyboardInterrupt: ignored

# New section

In [20]:
%%javascript
// Create the channel object
const channel = new BroadcastChannel('my-channel');

// Set up a listener
channel.onmessage = message => console.log('Received:', message.data);

// Post a message to all subscribers
channel.postMessage('Hello, world');

<IPython.core.display.Javascript object>

In [21]:
conn.disconnect()